## DataLoader
- [param] read dataset list, list of jpg
    -  assume has corresponding xml file beside
- [param] .name file for label hashing
- read bbbox data
- build dict [image path: annotation]


In [2]:
from torch.utils.data.dataset import Dataset

In [5]:
import random
import cv2
import sys
import numpy as np
import os
import matplotlib.pyplot as plt

from voc_io import *

%load_ext autoreload
%autoreload 2

In [40]:
class Detection_dataset(Dataset):
    def __init__(self, image_paths, cfg=None):
        super(Detection_dataset, self).__init__()
        
        self.cfg = cfg
        
        names = ["palm1_npt", "palm1_byt", "palm2_mtl"]
        self.names = {name:i for i,name in enumerate(names)}
        
        truth = {}
        f = open(image_paths, 'r', encoding='utf-8')
        for line in f.readlines():
            path = line.rstrip()
            truth[path] = self.read_annot(path)         
        self.truth = truth
        
    def read_annot(self, jpg):
        parts = jpg.split('.')
        parts[-1] = 'xml'
        xml = '.'.join(parts)
        
        annots = PascalVocReader(xml).getAnnot()
        for annot in annots:
            uid = self.names.get(annot[-1], None)
            if uid == None:
                uid = self.extend_name(annot[-1])
            annot[-1] = uid
        return annots
        
    def extend_name(self, name):
        newId = len(self.names.keys())
        self.names[name] = newId
        print("Added Missing Name: %d, %s"%(newId, name))
        return newId
        
    def __getitem__(self, index):
        img_path = list(self.truth.keys())[index]
        bboxes = np.array(self.truth.get(img_path), dtype=np.float)
#         img_path = os.path.join(self.cfg.dataset_dir, img_path)
    
        
    def __len__(self):
        return len(self.truth.keys())

In [41]:
data_list = "/Users/ckh/OneDrive - Default Directory/Hui_Wan/train.txt"
dataset = Detection_dataset(data_list)

In [44]:
dataset.truth

{'/Users/ckh/OneDrive - Default Directory/Hui_Wan/Sample_Anno_Data_BYT/10_gsge_op16b_0001.JPG': [[176,
   379,
   210,
   413,
   1],
  [527, 707, 287, 467, 1],
  [636, 861, 536, 761, 1],
  [794, 988, 821, 1000, 1],
  [461, 655, 821, 1000, 1],
  [345, 548, 516, 719, 1],
  [10, 199, 510, 699, 1],
  [125, 337, 796, 1000, 1],
  [846, 1000, 311, 480, 1]],
 '/Users/ckh/OneDrive - Default Directory/Hui_Wan/Sample_Anno_Data_BYT/10_gsge_op16b_0002.JPG': [[238,
   445,
   54,
   261,
   1],
  [556, 750, 103, 297, 1],
  [690, 862, 387, 559, 1],
  [363, 575, 354, 566, 1],
  [67, 247, 336, 516, 1],
  [183, 381, 614, 812, 1],
  [503, 697, 650, 844, 1],
  [827, 1000, 674, 859, 1],
  [863, 1000, 117, 297, 1]],
 '/Users/ckh/OneDrive - Default Directory/Hui_Wan/Sample_Anno_Data_BYT/10_gsge_op16b_0003.JPG': [[87,
   263,
   245,
   421,
   1],
  [392, 586, 245, 439, 1],
  [690, 897, 267, 474, 1],
  [847, 1000, 563, 730, 1],
  [532, 739, 507, 714, 1],
  [221, 410, 534, 723, 1],
  [45, 234, 774, 963, 1],
